### Depth-First Traversal

Depth-first search (DFS) is an uninformed search that systematically traverses nodes until it finds its goal. in the context of a tree, DFS involves descending down a child's child, iteratively, and then backtracking and turning to each of its siblings--in orther words, it prefers to go deep before going broad. In a graph there are no children or siblings, only neighbors. However, DFS can produce a *spanning tree* of the nodes it has visited. 

Here is the DFS algorithim, simplistically described:

- Start at some node $n$  
- Mark $n$ as visited  
- For each neighbor $n_i$  of $n$ where $n_i$ has not been visited, recursively apply DFS node $n_i$

#### Implementation

In [7]:
# a list of nodes in the order they were visited

def DFS_nodes(graph, node, visited=[]):
    visited.append(node)
    for neighbor in graph[node]:
        if not neighbor in visited:
            DFS_nodes(graph, neighbor, visited)
    return visited

# a list of edges in the order they were traversed. This does not include back edges

def DFS_edges(graph, node, visited=[], edges=[]):
    visited.append(node)
    for ni in graph[node]:
        if not ni in visited:
            edges.append((node, ni))
            DFS_edges(graph, ni, visited, edges)
    return edges

In [4]:
import networkx.generators.small as nx
g = nx.krackhardt_kite_graph()

In [5]:
DFS_nodes(g, 0)

[0, 1, 3, 2, 5, 6, 4, 7, 8, 9]

In [8]:
DFS_edges(g, 0)

[(0, 1), (1, 3), (3, 2), (2, 5), (5, 6), (6, 4), (6, 7), (7, 8), (8, 9)]

## DFS with NetworkX

NetworkX provides a DFS implementation by D. Eppstein

In [13]:
from networkx import traversal 

edges = traversal.dfs_edges(g)
edges

<generator object dfs_edges at 0x7fca86919430>

In [14]:
list(edges)

[(0, 1), (1, 3), (3, 2), (2, 5), (5, 6), (6, 4), (6, 7), (7, 8), (8, 9)]

We can get a more conventional tree view with a successor dictionary:

In [15]:
traversal.dfs_successors(g)

{0: [1], 1: [3], 3: [2], 2: [5], 5: [6], 6: [4, 7], 7: [8], 8: [9]}

In [16]:
# predecessors are available -- a view of the tree upside down
traversal.dfs_predecessors(g)

{1: 0, 3: 1, 2: 3, 5: 2, 6: 5, 4: 6, 7: 6, 8: 7, 9: 8}

In [17]:
# this is all just lists and dictionaries
traversal.dfs_successors(g)[3]

[2]

Finally, we can produce a directed graph representing the DFS traversal:

In [22]:
tree = traversal.dfs_tree(g)
tree

In [23]:
tree.successors(0)

In [24]:
tree.succ

AdjacencyView({0: {1: {}}, 1: {3: {}}, 2: {5: {}}, 3: {2: {}}, 4: {}, 5: {6: {}}, 6: {4: {}, 7: {}}, 7: {8: {}}, 8: {9: {}}, 9: {}})

### Breadth-First Traversal

Breadth-first traversal is the **opposite** of DFS -- it visits all of the immediate neighbors first and only then proceeds to their neighbors. 

#### Algorithm

The difference is the order in which they traverse encountered nodes. 

DFS will explore a new neighbor node immediately as it sees it whereas BFS must store each neighbor until it has seen them all, and only then descend down each one.

As a consequence, there is no natural recursive form for BFS.

#### BFS with NetworkX

The interface is similar to DFS, which one distinction: a starting node must be specified.

In [25]:
edges = traversal.bfs_edges(g, 0)
list(edges)

[(0, 1), (0, 2), (0, 3), (0, 5), (1, 4), (1, 6), (5, 7), (7, 8), (8, 9)]

In [26]:
tree = traversal.bfs_tree(g, 0)
tree

The difference between DFS and BFS traversal trees is clearly visible. 

In [31]:
traversal.bfs_successors(g, 0)

<generator object bfs_successors at 0x7fca866cdc80>

In [32]:
traversal.dfs_successors(g, 0)

{0: [1], 1: [3], 3: [2], 2: [5], 5: [6], 6: [4, 7], 7: [8], 8: [9]}

### Paths and Walks

A **walk** is an alternating sequence of nodes and edges that connect them. A walk is open if the starting and ending nodes are different and closed otherwise. The length of the walk is the number of edges. 

A **path** is an open simple (where no node is crossed twice) walk. A closed simple walk is a **cycle**. Formally, a path can have a length 0, where a cycle cannot. 

Paths and cycles are essential to understanding the structure of graphs and quantifying that with machines. NetworkX provides a number of path algorithms:

In [33]:
from networkx import algorithms

algorithms.shortest_path(g, 0, 5)

[0, 5]

In [34]:
algorithms.shortest_path(g, 0, 7)

[0, 5, 7]

In [35]:
algorithms.average_shortest_path_length(g)

1.9777777777777779

We can even list the shortest paths in a graph:

In [48]:
path = algorithms.all_pairs_shortest_path(g)
next(path)

(0,
 {0: [0],
  1: [0, 1],
  2: [0, 2],
  3: [0, 3],
  5: [0, 5],
  4: [0, 1, 4],
  6: [0, 1, 6],
  7: [0, 5, 7],
  8: [0, 5, 7, 8],
  9: [0, 5, 7, 8, 9]})

### Dijkstra's Algorithm

A special mention goes to a graph search algorithm published by Edsger Dijkstra in 1959. For a given vertext it finds the lowest cost path to all other vertices, where "cost" is determined by summing edge weights. In graphs where edge weights correspond to distance (in unweighted graphs the weights are assumed to be one) the found path is the shortest. The algorithm can also determine the lowest cost path between two given vertices: 

In [54]:
algorithms.dijkstra_path(g, 1, 5)

[1, 0, 5]

In [55]:
algorithms.dijkstra_predecessor_and_distance(g, 1, 5)

({1: [],
  0: [1],
  3: [1],
  4: [1],
  6: [1],
  2: [0, 3],
  5: [0, 3, 6],
  7: [6],
  8: [7],
  9: [8]},
 {1: 0, 0: 1, 3: 1, 4: 1, 6: 1, 2: 2, 5: 2, 7: 2, 8: 3, 9: 4})

We can compare the shortest path algorithms with relative ease. First we will need node pairs:

In [56]:
import itertools 

g.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9))

In [58]:
list(itertools.combinations(g.nodes(), 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 7),
 (6, 8),
 (6, 9),
 (7, 8),
 (7, 9),
 (8, 9)]

Note that because our graph is undirected we don't need to look at reverse paths, but even this may be a little too much. For brevity, we can limit the node space to, say, first 4 nodes, using Python's array slicing syntax: 

In [59]:
nn = g.nodes()
nn

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9))

In [61]:
list(nn)[:4]

[0, 1, 2, 3]

In [62]:
pairs = list(itertools.combinations(list(nn)[:4], 2))
pairs

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

Using another Python technique, argument array expansion, we can look at them side by side: 

In [64]:
for pair in itertools.combinations(list(nn)[:8], 2):
    print(algorithms.shortest_path(g, *pair)), 
    algorithms.dijkstra_path(g, *pair)

[0, 1]
[0, 2]
[0, 3]
[0, 1, 4]
[0, 5]
[0, 1, 6]
[0, 5, 7]
[1, 0, 2]
[1, 3]
[1, 4]
[1, 0, 5]
[1, 6]
[1, 6, 7]
[2, 3]
[2, 3, 4]
[2, 5]
[2, 3, 6]
[2, 5, 7]
[3, 4]
[3, 5]
[3, 6]
[3, 5, 7]
[4, 3, 5]
[4, 6]
[4, 6, 7]
[5, 6]
[5, 7]
[6, 7]


They are the same. But what about weights? We can throw some random weights at the graph and see what happens. We do this by constructing a list of node-node-weight triples from the list of node pairs. For our eyes sakes let us use integer weights between 1 and 10:

In [68]:
from random import choice

new_edges = [x + (choice(range(10)),) for x in g.edges()]
new_edges

[(0, 1, 7),
 (0, 2, 0),
 (0, 3, 0),
 (0, 5, 0),
 (1, 3, 6),
 (1, 4, 9),
 (1, 6, 4),
 (2, 3, 8),
 (2, 5, 3),
 (3, 4, 5),
 (3, 5, 3),
 (3, 6, 7),
 (4, 6, 2),
 (5, 6, 1),
 (5, 7, 5),
 (6, 7, 0),
 (7, 8, 6),
 (8, 9, 2)]

In [69]:
g.clear()
g.add_weighted_edges_from(new_edges)

Now we can compare the paths again:

In [79]:
for pair in itertools.combinations(list(nn)[:8], 2):
    print(algorithms.shortest_path(g, *pair)), 
    algorithms.dijkstra_path(g, *pair)

[0, 1]
[0, 2]
[0, 3]
[0, 5]
[0, 1, 4]
[0, 1, 6]
[0, 5, 7]
[1, 0, 2]
[1, 3]
[1, 0, 5]
[1, 4]
[1, 6]
[1, 6, 7]
[2, 3]
[2, 5]
[2, 3, 4]
[2, 3, 6]
[2, 5, 7]
[3, 5]
[3, 4]
[3, 6]
[3, 5, 7]
[5, 3, 4]
[5, 6]
[5, 7]
[4, 6]
[4, 6, 7]
[6, 7]


### Graph Distance

Graph distance can be measured a number of ways:

**Shortest path (unweighted graph)**
- The simplest measure of distance; the number of edges one must walk from A to B

**Cost-based shortest path (weighted graph)**  
- In a *weighted* graph, every edge has an associated value, called *weight*. This value could, for example, represent the physical distance between two points on a topographic map. Naturally, the distance from A to B will be the sum of distances between the intermediate points. The choice for the shortest path is the one with the least combined distance, but not necessarily the fewest nodes. Often we give this weight-as-a-distance value the more general term "cost", meaning it will cost us this many CPU cycles/hops/hours/etc. to travel the path. 